In [3]:
## importing modules and loading the data
import numpy as np
import os
import nibabel as nib
import matplotlib as mlp
import MainFunctions as MF
import bokeh
import bokeh.plotting as bpl
from bokeh.models import ColorBar, BasicTicker, LinearColorMapper
import wav as wav
# downloading the data
path0=os.getcwd()



In [15]:
# download the data if it doesnt exist. 
path=path0+'/content/bold5000/sub-CSI3_anat/sub-CSI3_ses-16_T1w.nii.gz'
downloaded=os.path.isfile(path)
if not downloaded:
    !pip install awscli
    os.chdir(path0)
    os.mkdir('./content')
    os.chdir('./content')
    os.mkdir('./bold5000')
    os.chdir(path0+'/content/bold5000')
    !aws s3 sync --no-sign-request s3://openneuro.org/ds001499/sub-CSI3/ses-16/anat/ ./sub-CSI3_anat/
    os.chdir(path0)
# load the data
data,hdr= MF.load_data()

shell-init: error retrieving current directory: getcwd: cannot access parent directories: Operation not permitted
The folder you are executing pip from can no longer be found.
download: s3://openneuro.org/ds001499/sub-CSI3/ses-16/anat/sub-CSI3_ses-16_T1w.json to sub-CSI3_anat/sub-CSI3_ses-16_T1w.json
download: s3://openneuro.org/ds001499/sub-CSI3/ses-16/anat/tmp_dcm2bids/log/sub-CSI3_20180607T112621.log to sub-CSI3_anat/tmp_dcm2bids/log/sub-CSI3_20180607T112621.log
download: s3://openneuro.org/ds001499/sub-CSI3/ses-16/anat/sub-CSI3_ses-16_T1w.nii.gz to sub-CSI3_anat/sub-CSI3_ses-16_T1w.nii.gz


In [17]:
# looking at a sample slice of the data
Nx = data.shape[0]
Ny = data.shape[1]
Nz = data.shape[2]
n = Ny
X = data[Nx//2, :, :];
bpl.output_notebook()
MF.imagesc(data[Nx//2, :, :], title='MR Image We Will Recover')

Loading BokehJS ...

Figure(id='1003', ...)